Converting raw `html` to `vdom`.

In [2]:
import html, vdom.svg, functools, inspect

`dom` are a collection or existing `vdom` tags; if we can't find a tag then we make our won `vdom.createComponent`.

In [3]:
dom = {**vars(vdom), **vars(vdom.svg)}

`get_tag` will return the `'tag_name'` inside of a `functools.partial`.

In [104]:
def get_tag(partial):
    if not callable(partial): return None
    def get_func(f): 
        return get_func(f.func) if hasattr(f, 'func') else f
    return inspect.getclosurevars(get_func(partial)).nonlocals['tag_name']

In [207]:
from html.parser import HTMLParser

class vdomParser(HTMLParser):
    buf = None    
    def __call__(self, str):
        self.buf = None
        return self.feed(str) or list(filter(bool, self.buf))
    
    def handle_starttag(self, tag, attrs): 
        self.buf = self.buf or [[]]
        tag = dom.get(tag, vdom.create_component(tag))
        self.buf[-1] = self.buf[-1] or [tag]
        self.buf[-1].append(functools.partial(tag, **dict(attrs)))

    def handle_endtag(self, tag):
        object, parent = self.buf[-1].pop(), self.buf[-1] and self.buf[-1].pop() or None
        if tag == get_tag(object): 
            object = object()
            if not self.buf[-1]:
                self.buf[-1] = object
                return self.buf.append([])
        if self.buf[-1]: object = functools.partial(parent, object)
        else: object = functools.partial(object, F"</{tag}>")
        self.buf[-1].append(object)     
    
    def handle_data(self, data): 
        if self.buf[-1]: self.buf[-1].append(functools.partial(self.buf[-1].pop(), data))
        else: self.buf.extend([data, []])
            
parser = vdomParser()

In [208]:
    sample = vdom.div(
        vdom.h1(vdom.code('vdom'), ' is sweet.'),
        vdom.p('I don''t think there is a raw ', vdom.code('html'), ' object to ', vdom.code('vdom'), ' yet.'),
        vdom.p('This post attempts to think about how that is possible')
    )

In [209]:
    def test_transform():
        assert parser(
            sample._repr_html_()
        )[0]._repr_html_() == sample._repr_html_()
    __name__ == '__main__'  and test_transform()

In [210]:
    def test_others():
        parser("<h1>testing</p></h1>")

In [204]:
    def test_graphviz(): 
        import graphviz
        vdomParser()(graphviz.Source('graph {A}')._repr_svg_())

In [205]:
    if __name__ == '__main__':
        !ipython -m pytest -- 2018-11-01-vdom-from-raw-html.ipynb

============================= test session starts =============================
platform win32 -- Python 3.6.6, pytest-3.5.1, py-1.5.3, pluggy-0.6.0
Matplotlib: 2.2.2
Freetype: 2.8.1
rootdir: C:\Users\deathbeds\deathbeds.github.io\deathbeds, inifile: tox.ini
plugins: xonsh-0.8.1, xdist-1.22.5, testmon-0.9.12, remotedata-0.2.1, parallel-0.0.2, openfiles-0.3.0, mpl-0.9, localserver-0.4.1, forked-0.2, doctestplus-0.1.3, arraydiff-0.2, hypothesis-3.66.16, importnb-0.5.0
collected 3 items

2018-11-01-vdom-from-raw-html.ipynb ...                                  [100%]

========================== 3 passed in 0.55 seconds ===========================
